## Imports

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
login(user_secrets.get_secret("HUGGINGFACE_TOKEN"))

## Model & Tokenizer

In [2]:
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Few-Shot Prompt Template

In [3]:
# Few-shot prompt
def build_prompt(review):
    return f"""
Classify the sentiment of the following review as Positive, Neutral, or Negative.

Review: "This product is terrible and broke after one use."
Sentiment: Negative

Review: "It works as expected, nothing special."
Sentiment: Neutral

Review: "I love this! It’s amazing and works perfectly."
Sentiment: Positive

Review: "{review}"
Sentiment:"""

## Classification Function

In [4]:
def classify_review(review, max_new_tokens=10):
    prompt = build_prompt(review)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    after_sentiment = generated_text.split("Sentiment:")[-1].strip()
    # Stop at the first line break (or anything resembling a new Review)
    sentiment = after_sentiment.split("\n")[0].split("Review:")[0].strip()
    return sentiment

## Reviews & Predictions

In [5]:
text_list = ["I wasn't expecting much, but this exceeded my expectations." 
             "The build quality feels premium, and the instructions were "
             "easy to follow. Setup took under 10 minutes, and it's been "
             "working flawlessly ever since. I've used it daily without "
             "any issues. Definitely worth the price and would happily "
             "recommend it to anyone looking for a reliable option in this category.", # Positive

             "It works as described, but there's nothing particularly "
             "special about it. The performance is fine for basic tasks, "
             "though I did notice some minor lag during more demanding use. "
             "Packaging was a bit flimsy, but the item arrived intact. "
             "If you're looking for something simple and functional, "
             "this will do, just don't expect premium quality or standout features.", # Neutral

             "Pretty disappointed overall. The product didn't match the "
             "pictures and felt cheaply made. It stopped working properly "
             "after just a few uses, and customer service was slow to respond. "
             "I tried troubleshooting it myself, but nothing helped. For the price, "
             "I expected a much better experience. Wouldn't buy again, and I'd "
             "recommend looking elsewhere if you need something dependable.", # Negative

             "I'm obsessed with this beauty product! 💖 From the first use, I noticed a real difference "
             "— my skin feels smoother, looks brighter, and has that natural glow ✨ I’ve gotten so many "
             "compliments lately, and it’s all thanks to this little miracle in a bottle 🧴💫 "
             "The texture is silky, it absorbs quickly, and the scent is light and refreshing 🌸 "
             "Plus, it layers perfectly under makeup without pilling 🙌 10/10 would recommend to "
             "anyone looking to elevate their skincare game 💆‍♀️💕 "
             f"#GlowUp #SkincareGoals #HolyGrailProduct 💎🌟",

             "Unfortunately, this product didn’t work for me 😕 I had high hopes, "
             "but after a few uses, my skin started feeling dry and irritated 😣💔 "
             "The texture felt heavy and didn’t absorb well — it just sat on my skin like a "
             "sticky layer 🧴😬 And the scent? Way too strong for something that’s supposed to be gentle 🌪️👃 "
             "I really wanted to love it, but sadly, it’s a no from me 🙅‍♀️ Wouldn’t repurchase. "
             f"#SkincareMiss #NotForMe #Disappointed 😞🚫",

            "This product was just okay for me — not amazing, but not terrible either ¯\(ツ)/¯ "
            "It did a decent job moisturizing, and the packaging is really cute ✨📦 "
            "However, I didn’t notice any dramatic changes in my skin 🧴=🤷‍♀️ "
            
            "The texture is nice and lightweight, and it has a mild scent which I appreciate 🌿 "
            "If you’re looking for something basic that gets the job done, this might be it ✅ "
            "Just don’t expect any ✨magic✨ results. "
            f"#SkincareReview #MehMoment #MildVibes"
            ]

In [6]:
for text in text_list:
    print(f"- Text: {text}")
    prediction = classify_review(text)
    print("- Predicted Sentiment:", prediction, "\n")

- Text: I wasn't expecting much, but this exceeded my expectations.The build quality feels premium, and the instructions were easy to follow. Setup took under 10 minutes, and it's been working flawlessly ever since. I've used it daily without any issues. Definitely worth the price and would happily recommend it to anyone looking for a reliable option in this category.
- Predicted Sentiment: Positive 

- Text: It works as described, but there's nothing particularly special about it. The performance is fine for basic tasks, though I did notice some minor lag during more demanding use. Packaging was a bit flimsy, but the item arrived intact. If you're looking for something simple and functional, this will do, just don't expect premium quality or standout features.
- Predicted Sentiment: Neutral 

- Text: Pretty disappointed overall. The product didn't match the pictures and felt cheaply made. It stopped working properly after just a few uses, and customer service was slow to respond. I tr